In [ ]:
!pip install pyiqa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 31.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 19.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.2/276.2 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 133.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.4/299.4 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.4/226.4 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Part 0: Environment Setup & Drive Mounting
from google.colab import drive
import os

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Define the project root directory in your Drive
PROJECT_ROOT = "/content/drive/My Drive/ECE253_Project"

# 3. Define sub-directories for organized storage
DIRS = {
    "GT": os.path.join(PROJECT_ROOT, "1_GroundTruth"),      # Folder for Sharp images
    "BLUR": os.path.join(PROJECT_ROOT, "2_InputBlur"),      # Folder for Blurred images
    "RL_OUT": os.path.join(PROJECT_ROOT, "3_Output_RL"),    # Folder for RL results
    "NAF_OUT": os.path.join(PROJECT_ROOT, "4_Output_NAFNet"), # Folder for NAFNet results
    "NAF_FT_OUT": os.path.join(PROJECT_ROOT, "5_Output_NAFNet_FineTuned"),
    "METRICS": os.path.join(PROJECT_ROOT, "Metrics")        # Folder for CSV files
}

# 4. Create directories if they don't exist
for k, v in DIRS.items():
    os.makedirs(v, exist_ok=True)

print(f"✅ Working directory ready: {PROJECT_ROOT}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Working directory ready: /content/drive/My Drive/ECE253_Project


In [ ]:
# Cell 3.1: NAFNet Architecture Definition
import torch
import torch.nn as nn
import torch.nn.functional as F

# --- Layer Normalization ---
class LayerNormFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, weight, bias, eps):
        ctx.eps = eps
        N, C, H, W = x.size()
        mu = x.mean(1, keepdim=True)
        var = (x - mu).pow(2).mean(1, keepdim=True)
        y = (x - mu) / (var + eps).sqrt()
        ctx.save_for_backward(y, var, weight)
        y = weight.view(1, C, 1, 1) * y + bias.view(1, C, 1, 1)
        return y

    @staticmethod
    def backward(ctx, grad_output):
        eps = ctx.eps
        N, C, H, W = grad_output.size()
        y, var, weight = ctx.saved_tensors
        g = grad_output * weight.view(1, C, 1, 1)
        mean_g = g.mean(dim=1, keepdim=True)
        mean_gy = (g * y).mean(dim=1, keepdim=True)
        gx = 1. / torch.sqrt(var + eps) * (g - y * mean_gy - mean_g)
        return gx, (grad_output * y).sum(dim=3).sum(dim=2).sum(dim=0), grad_output.sum(dim=3).sum(dim=2).sum(dim=0), None

class LayerNorm2d(nn.Module):
    def __init__(self, channels, eps=1e-6):
        super(LayerNorm2d, self).__init__()
        self.register_parameter('weight', nn.Parameter(torch.ones(channels)))
        self.register_parameter('bias', nn.Parameter(torch.zeros(channels)))
        self.eps = eps
    def forward(self, x):
        return LayerNormFunction.apply(x, self.weight, self.bias, self.eps)

# --- SimpleGate ---
class SimpleGate(nn.Module):
    def forward(self, x):
        x1, x2 = x.chunk(2, dim=1)
        return x1 * x2

# --- NAFBlock ---
class NAFBlock(nn.Module):
    def __init__(self, c, DW_Expand=2, FFN_Expand=2, drop_out_rate=0.):
        super().__init__()
        dw_channel = c * DW_Expand
        self.conv1 = nn.Conv2d(in_channels=c, out_channels=dw_channel, kernel_size=1, padding=0, stride=1, groups=1, bias=True)
        self.conv2 = nn.Conv2d(in_channels=dw_channel, out_channels=dw_channel, kernel_size=3, padding=1, stride=1, groups=dw_channel, bias=True)
        self.conv3 = nn.Conv2d(in_channels=dw_channel // 2, out_channels=c, kernel_size=1, padding=0, stride=1, groups=1, bias=True)
        self.sca = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels=dw_channel // 2, out_channels=dw_channel // 2, kernel_size=1, padding=0, stride=1, groups=1, bias=True),
        )
        self.conv4 = nn.Conv2d(in_channels=c, out_channels=c * FFN_Expand, kernel_size=1, padding=0, stride=1, groups=1, bias=True)
        self.conv5 = nn.Conv2d(in_channels=c * FFN_Expand // 2, out_channels=c, kernel_size=1, padding=0, stride=1, groups=1, bias=True)
        self.norm1 = LayerNorm2d(c)
        self.norm2 = LayerNorm2d(c)
        self.dropout1 = nn.Dropout(drop_out_rate) if drop_out_rate > 0. else nn.Identity()
        self.dropout2 = nn.Dropout(drop_out_rate) if drop_out_rate > 0. else nn.Identity()
        self.beta = nn.Parameter(torch.zeros((1, c, 1, 1)), requires_grad=True)
        self.gamma = nn.Parameter(torch.zeros((1, c, 1, 1)), requires_grad=True)

    def forward(self, x):
        inp = x
        x = self.norm1(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = SimpleGate()(x)
        x = x * self.sca(x)
        x = self.conv3(x)
        x = self.dropout1(x)
        y = inp + x * self.beta
        x = self.norm2(y)
        x = self.conv4(x)
        x = SimpleGate()(x)
        x = self.conv5(x)
        x = self.dropout2(x)
        return y + x * self.gamma

# --- NAFNet ---
class NAFNet(nn.Module):
    def __init__(self, img_channel=3, width=16, middle_blk_num=1, enc_blk_nums=[], dec_blk_nums=[]):
        super().__init__()
        self.intro = nn.Conv2d(in_channels=img_channel, out_channels=width, kernel_size=3, padding=1, stride=1, groups=1, bias=True)
        self.ending = nn.Conv2d(in_channels=width, out_channels=img_channel, kernel_size=3, padding=1, stride=1, groups=1, bias=True)
        self.encoders = nn.ModuleList()
        self.decoders = nn.ModuleList()
        self.middle_blks = nn.ModuleList()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        chan = width
        for num in enc_blk_nums:
            self.encoders.append(nn.Sequential(*[NAFBlock(chan) for _ in range(num)]))
            self.downs.append(nn.Conv2d(chan, 2*chan, 2, 2))
            chan = chan * 2
        self.middle_blks = nn.Sequential(*[NAFBlock(chan) for _ in range(middle_blk_num)])
        for num in dec_blk_nums:
            self.ups.append(nn.Sequential(nn.Conv2d(chan, chan * 2, 1, bias=False), nn.PixelShuffle(2)))
            chan = chan // 2
            self.decoders.append(nn.Sequential(*[NAFBlock(chan) for _ in range(num)]))

    def forward(self, x):
        inp = x
        x = self.intro(x)
        encs = []
        for encoder, down in zip(self.encoders, self.downs):
            x = encoder(x)
            encs.append(x)
            x = down(x)
        x = self.middle_blks(x)
        for decoder, up, enc_skip in zip(self.decoders, self.ups, encs[::-1]):
            x = up(x)
            x = x + enc_skip
            x = decoder(x)
        x = self.ending(x)
        return x + inp

print("✅ Cell 3.1: NAFNet architecture defined successfully.")

✅ Cell 3.1: NAFNet architecture defined successfully.


In [ ]:
# Cell 3.2: Load Pre-trained Weights
import os
import torch

# 1. Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f">>> Running on device: {device}")

# 2. Initialize Model (Width=32 configuration for GoPro)
model_naf = NAFNet(img_channel=3, width=32, middle_blk_num=1,
                   enc_blk_nums=[1, 1, 1, 28], dec_blk_nums=[1, 1, 1, 1])

# 3. Download and Load Weights
weight_path = "NAFNet-GoPro-width32.pth"
weight_url = "https://huggingface.co/nyanko7/nafnet-models/resolve/main/NAFNet-GoPro-width32.pth"

if not os.path.exists(weight_path):
    print(">>> Downloading pre-trained weights from HuggingFace...")
    os.system(f"curl -L -o {weight_path} {weight_url}")

print(">>> Loading weights into model...")
try:
    checkpoint = torch.load(weight_path, map_location=device)
    param_dict = checkpoint['params'] if 'params' in checkpoint else checkpoint
    model_naf.load_state_dict(param_dict, strict=False)
    model_naf.to(device)
    # Set to Eval mode immediately since we are not training
    model_naf.eval()
    print("✅ Cell 3.2: Model loaded and set to Eval mode.")
except Exception as e:
    print(f"❌ Error loading weights: {e}")

>>> Running on device: cuda
>>> Downloading pre-trained weights from HuggingFace...
>>> Loading weights into model...
✅ Cell 3.2: Model loaded and set to Eval mode.


In [ ]:
# Cell 3.3: NAFNet Inference - Updated with NIQE & No-GT Support
# ==============================================================
import pandas as pd
from skimage import metrics
import cv2
import numpy as np
from tqdm import tqdm
import os
import torch
import pyiqa

# 1. Initialize NIQE
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(">>> Loading NIQE metric...")
try:
    niqe_metric = pyiqa.create_metric('niqe', device=device)
except Exception as e:
    print(f"⚠️ Warning: Could not load NIQE: {e}")
    niqe_metric = None

# Ensure model is in eval mode
model_naf.eval()
results_naf = []

files = sorted(os.listdir(DIRS["BLUR"]))
print(f">>> Starting NAFNet Inference on {len(files)} images...")

for f in tqdm(files, desc="NAFNet Inference"):
    # Paths
    path_blur = os.path.join(DIRS["BLUR"], f)
    path_gt = os.path.join(DIRS["GT"], f)
    # Output path (using NAF_OUT as default)
    path_out = os.path.join(DIRS["NAF_OUT"], f)

    # Check GT
    has_gt = os.path.exists(path_gt)

    # Read Image
    img_blur_orig = cv2.imread(path_blur)
    if img_blur_orig is None: continue # Skip corrupt files
    img_blur_orig = cv2.cvtColor(img_blur_orig, cv2.COLOR_BGR2RGB)

    # Preprocess
    img_tensor = torch.from_numpy(img_blur_orig.astype(np.float32)/255.0)
    img_tensor = img_tensor.permute(2,0,1).unsqueeze(0).to(device)

    # Inference
    with torch.no_grad():
        # Handle Padding (NAFNet requires input to be multiple of 32)
        _, _, h, w = img_tensor.shape
        h_n = (h // 32) * 32
        w_n = (w // 32) * 32
        inp = img_tensor[:, :, :h_n, :w_n]

        output = model_naf(inp)

    # Post-process
    out_img = output.squeeze().cpu().permute(1,2,0).numpy()
    out_img = np.clip(out_img, 0, 1)
    out_uint8 = (out_img * 255.0).astype(np.uint8)

    # Save Result
    cv2.imwrite(path_out, cv2.cvtColor(out_uint8, cv2.COLOR_RGB2BGR))

    # --- Metrics Calculation ---
    row = {"Image": f, "Method": "NAFNet", "Has_GT": has_gt}

    # 1. NIQE (No-Reference)
    if niqe_metric:
        out_tensor = torch.from_numpy(out_uint8).permute(2,0,1).unsqueeze(0).float() / 255.0
        row["NIQE"] = niqe_metric(out_tensor.to(device)).item()
    else:
        row["NIQE"] = None

    # 2. PSNR/SSIM (Full-Reference)
    if has_gt:
        img_gt = cv2.cvtColor(cv2.imread(path_gt), cv2.COLOR_BGR2RGB)
        # Important: Crop GT to match NAFNet output (due to padding)
        gt_crop = img_gt[:h_n, :w_n, :]

        row["PSNR"] = metrics.peak_signal_noise_ratio(gt_crop, out_uint8)
        row["SSIM"] = metrics.structural_similarity(gt_crop, out_uint8, channel_axis=2, win_size=3)
    else:
        row["PSNR"] = None
        row["SSIM"] = None

    results_naf.append(row)

# Save Metrics
csv_path = os.path.join(DIRS["METRICS"], "metrics_nafnet.csv")
pd.DataFrame(results_naf).to_csv(csv_path, index=False)

print(f"✅ Cell 3.3 Complete: Processed {len(files)} images.")
print(f"✅ Metrics saved to {csv_path}")

>>> Loading NIQE metric...
Downloading: "https://huggingface.co/chaofengc/IQA-PyTorch-Weights/resolve/main/niqe_modelparameters.mat" to /root/.cache/torch/hub/pyiqa/niqe_modelparameters.mat



100%|██████████| 8.15k/8.15k [00:00<00:00, 31.9MB/s]


>>> Starting NAFNet Inference on 110 images...


NAFNet Inference: 100%|██████████| 110/110 [03:36<00:00,  1.96s/it]


✅ Cell 3.3 Complete: Processed 110 images.
✅ Metrics saved to /content/drive/My Drive/ECE253_Project/Metrics/metrics_nafnet.csv
